In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Numerical and data manipulation libraries
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib
matplotlib.use('Agg')  # Ensures that matplotlib does not use any Xwindows backend
import matplotlib.pyplot as plt
plt.switch_backend('Agg')  # Switch backend if only plt is imported
%matplotlib inline

import seaborn as sns

# Scikit-learn components for modeling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import LabelEncoder

# Print statement to show successful imports
print('Libraries imported successfully.')

# Data Loading and Inspection

We load the synthetic DNA dataset from the CSV file and examine its structure. The file is assumed to be located in the same directory as this notebook.

In [ ]:
# Loading the dataset
data_path = '/kaggle/input/dna-classification-dataset/synthetic_dna_dataset.csv'
df = pd.read_csv(data_path, encoding='ascii', delimiter=',')

# Display basic information about the dataset
print('Dataset shape:', df.shape)
print('Dataset columns:', df.columns.tolist())
df.head()

# Data Cleaning and Preprocessing

In this section, we check for missing values, ensure that the data types are correct, and handle potential data inconsistencies. Since we have date information in some datasets, it is important to infer the type if needed; however, our current dataset does not appear to include date fields.

In [ ]:
# Checking for missing values
missing_values = df.isnull().sum()
print('Missing values in each column:')
print(missing_values)

# For this dataset, we assume no critical missing values. If there were missing values, one might consider imputation methods.

# Converting dtypes if necessary
# Example: if any date fields were present, one would use pd.to_datetime. Not applicable in this dataset.

# Confirming data types
print('Data Types:')
print(df.dtypes)

# Exploratory Data Analysis

Our exploratory data analysis (EDA) will involve a variety of visualizations to understand the structure and relationships within the data. We will use histograms, box plots, pair plots, and count plots to get a comprehensive view. In addition, we create a numeric correlation heatmap if the dataset contains four or more numeric columns.

In [ ]:
# Requirements (install if needed):
# pip install scikit-learn lightgbm catboost optuna imbalanced-learn shap

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import optuna
import lightgbm as lgb
from catboost import CatBoostClassifier
import shap
import warnings
warnings.filterwarnings('ignore')

# ---- 1) Basic checks & label encode target ----
features = [
    'GC_Content', 'AT_Content', 'Sequence_Length',
    'Num_A', 'Num_T', 'Num_C', 'Num_G',
    'kmer_3_freq', 'Mutation_Flag'
]

missing_features = set(features) - set(df.columns.tolist())
if missing_features:
    raise ValueError(f'Missing features: {missing_features}')

# Encode target and save mapping
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Disease_Risk_Encoded'] = le.fit_transform(df['Disease_Risk'])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)

# ---- 2) Feature engineering functions ----
def add_engineered_features(X):
    X = X.copy()
    # safe GC/AT ratio (avoid divide by zero)
    X['GC_AT_ratio'] = X['GC_Content'] / (X['AT_Content'] + 1e-8)
    # per-base frequencies if sequence length is available
    X['A_freq'] = X['Num_A'] / (X['Sequence_Length'] + 1e-8)
    X['T_freq'] = X['Num_T'] / (X['Sequence_Length'] + 1e-8)
    X['C_freq'] = X['Num_C'] / (X['Sequence_Length'] + 1e-8)
    X['G_freq'] = X['Num_G'] / (X['Sequence_Length'] + 1e-8)
    # log transform of sequence length to reduce skew
    X['log_Sequence_Length'] = np.log1p(X['Sequence_Length'])
    return X

engineer_transformer = FunctionTransformer(add_engineered_features, validate=False)

# ---- 3) Columns types ----
numeric_cols = [
    'GC_Content', 'AT_Content', 'Sequence_Length',
    'Num_A', 'Num_T', 'Num_C', 'Num_G',
    'kmer_3_freq', 'GC_AT_ratio', 'A_freq', 'T_freq', 'C_freq', 'G_freq', 'log_Sequence_Length'
]
categorical_cols = ['Mutation_Flag']  # adjust if Mutation_Flag has more categories

# ---- 4) Preprocessing pipeline ----
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
], remainder='drop')  # engineered columns must exist before this step

# ---- 5) Train/test split ----
X = df[features].copy()
X = engineer_transformer.transform(X)  # add engineered columns
y = df['Disease_Risk_Encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---- 6) Baseline pipeline with RandomForest (with SMOTE to balance classes) ----
rf_pipeline = ImbPipeline([
    ('preproc', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('clf', RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced'))
])

# Quick CV baseline
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_pipeline, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
print("RF CV accuracy:", scores.mean(), "std:", scores.std())

# Fit & evaluate baseline
rf_pipeline.fit(X_train, y_train)
y_pred = rf_pipeline.predict(X_test)
print("Baseline RF Test:", classification_report(y_test, y_pred, target_names=le.classes_))

# ---- 7) Optuna optimization for LightGBM (recommended path) ----
def objective_lgb(trial):
    param = {
        'objective': 'multiclass' if len(np.unique(y)) > 2 else 'binary',
        'num_class': len(np.unique(y)) if len(np.unique(y)) > 2 else 1,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 16, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'verbosity': -1,
        'seed': 42,
        'n_jobs': -1,
    }

    # build pipeline for optuna (include smote inside training folds)
    lgb_clf = lgb.LGBMClassifier(n_estimators=1000, **param)

    pipe = ImbPipeline([
        ('preproc', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('clf', lgb_clf)
    ])

    # use cross_val_score on training set with early stopping (handled by LGBM's n_estimators)
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='f1_weighted', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize', study_name='lgb_opt')
study.optimize(objective_lgb, n_trials=50, show_progress_bar=True)  # increase trials as budget allows
print("Best LGB params:", study.best_params, "Best F1:", study.best_value)

# ---- 8) Train final LGB model on whole training set with best params ----
best_params = study.best_params
best_params.update({'n_estimators': 2000})  # allow more trees; use early stopping via cv if desired

final_lgb = lgb.LGBMClassifier(**best_params, random_state=42, n_jobs=-1)
final_pipe = ImbPipeline([
    ('preproc', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('clf', final_lgb)
])

final_pipe.fit(X_train, y_train)
y_pred = final_pipe.predict(X_test)
print("Final LGB Test report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# ---- 9) Optional: SHAP for interpretability (on the model after preprocessing) ----
# Note: SHAP needs the model and preprocessed feature names.
# Get transformed train features for shap
X_train_trans = preprocessor.fit_transform(X_train)  # careful: preprocessor was fit in pipeline; for SHAP we refit here
feature_names_num = numeric_cols
# combine ohe categories
ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
ohe_feature_names = list(ohe.get_feature_names_out(categorical_cols))
feature_names = feature_names_num + ohe_feature_names

explainer = shap.Explainer(final_pipe.named_steps['clf'])
# You might need a subset; SHAP on large dataset can be slow
shap_values = explainer(preprocessor.transform(X_test))
shap.summary_plot(shap_values, features=preprocessor.transform(X_test), feature_names=feature_names)

# ---- 10) Save models & encoders if needed ----
import joblib
joblib.dump(final_pipe, 'final_model_pipeline.joblib')
joblib.dump(le, 'label_encoder.joblib')

print("Saved final pipeline and label encoder.")

# Feature Engineering & Disease Risk Predictor

In this section we consider possible feature engineering steps. One natural transformation is to encode categorical variables into numerical values. We identify potential features for predicting the target variable, Disease_Risk. For instance, features related to sequence composition (GC_Content, AT_Content, nucleotide counts, etc.) can be used directly.

In [ ]:
pip install --upgrade scikit-learn imbalanced-learn

In [ ]:
# Requirements (install if needed):
# pip install scikit-learn lightgbm catboost optuna imbalanced-learn shap

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import optuna
import lightgbm as lgb
from catboost import CatBoostClassifier
import shap
import warnings
warnings.filterwarnings('ignore')

# ---- 1) Basic checks & label encode target ----
features = [
    'GC_Content', 'AT_Content', 'Sequence_Length',
    'Num_A', 'Num_T', 'Num_C', 'Num_G',
    'kmer_3_freq', 'Mutation_Flag'
]

missing_features = set(features) - set(df.columns.tolist())
if missing_features:
    raise ValueError(f'Missing features: {missing_features}')

# Encode target and save mapping
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Disease_Risk_Encoded'] = le.fit_transform(df['Disease_Risk'])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_mapping)

# ---- 2) Feature engineering functions ----
def add_engineered_features(X):
    X = X.copy()
    # safe GC/AT ratio (avoid divide by zero)
    X['GC_AT_ratio'] = X['GC_Content'] / (X['AT_Content'] + 1e-8)
    # per-base frequencies if sequence length is available
    X['A_freq'] = X['Num_A'] / (X['Sequence_Length'] + 1e-8)
    X['T_freq'] = X['Num_T'] / (X['Sequence_Length'] + 1e-8)
    X['C_freq'] = X['Num_C'] / (X['Sequence_Length'] + 1e-8)
    X['G_freq'] = X['Num_G'] / (X['Sequence_Length'] + 1e-8)
    # log transform of sequence length to reduce skew
    X['log_Sequence_Length'] = np.log1p(X['Sequence_Length'])
    return X

engineer_transformer = FunctionTransformer(add_engineered_features, validate=False)

# ---- 3) Columns types ----
numeric_cols = [
    'GC_Content', 'AT_Content', 'Sequence_Length',
    'Num_A', 'Num_T', 'Num_C', 'Num_G',
    'kmer_3_freq', 'GC_AT_ratio', 'A_freq', 'T_freq', 'C_freq', 'G_freq', 'log_Sequence_Length'
]
categorical_cols = ['Mutation_Flag']  # adjust if Mutation_Flag has more categories

# ---- 4) Preprocessing pipeline ----
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
], remainder='drop')  # engineered columns must exist before this step

# ---- 5) Train/test split ----
X = df[features].copy()
X = engineer_transformer.transform(X)  # add engineered columns
y = df['Disease_Risk_Encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---- 6) Baseline pipeline with RandomForest (with SMOTE to balance classes) ----
rf_pipeline = ImbPipeline([
    ('preproc', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('clf', RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced'))
])

# Quick CV baseline
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_pipeline, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
print("RF CV accuracy:", scores.mean(), "std:", scores.std())

# Fit & evaluate baseline
rf_pipeline.fit(X_train, y_train)
y_pred = rf_pipeline.predict(X_test)
print("Baseline RF Test:", classification_report(y_test, y_pred, target_names=le.classes_))

# ---- 7) Optuna optimization for LightGBM (recommended path) ----
def objective_lgb(trial):
    param = {
        'objective': 'multiclass' if len(np.unique(y)) > 2 else 'binary',
        'num_class': len(np.unique(y)) if len(np.unique(y)) > 2 else 1,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 16, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'verbosity': -1,
        'seed': 42,
        'n_jobs': -1,
    }

    # build pipeline for optuna (include smote inside training folds)
    lgb_clf = lgb.LGBMClassifier(n_estimators=1000, **param)

    pipe = ImbPipeline([
        ('preproc', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('clf', lgb_clf)
    ])

    # use cross_val_score on training set with early stopping (handled by LGBM's n_estimators)
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='f1_weighted', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize', study_name='lgb_opt')
study.optimize(objective_lgb, n_trials=50, show_progress_bar=True)  # increase trials as budget allows
print("Best LGB params:", study.best_params, "Best F1:", study.best_value)

# ---- 8) Train final LGB model on whole training set with best params ----
best_params = study.best_params
best_params.update({'n_estimators': 2000})  # allow more trees; use early stopping via cv if desired

final_lgb = lgb.LGBMClassifier(**best_params, random_state=42, n_jobs=-1)
final_pipe = ImbPipeline([
    ('preproc', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('clf', final_lgb)
])

final_pipe.fit(X_train, y_train)
y_pred = final_pipe.predict(X_test)
print("Final LGB Test report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# ---- 9) Optional: SHAP for interpretability (on the model after preprocessing) ----
# Note: SHAP needs the model and preprocessed feature names.
# Get transformed train features for shap
X_train_trans = preprocessor.fit_transform(X_train)  # careful: preprocessor was fit in pipeline; for SHAP we refit here
feature_names_num = numeric_cols
# combine ohe categories
ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
ohe_feature_names = list(ohe.get_feature_names_out(categorical_cols))
feature_names = feature_names_num + ohe_feature_names

explainer = shap.Explainer(final_pipe.named_steps['clf'])
# You might need a subset; SHAP on large dataset can be slow
shap_values = explainer(preprocessor.transform(X_test))
shap.summary_plot(shap_values, features=preprocessor.transform(X_test), feature_names=feature_names)

# ---- 10) Save models & encoders if needed ----
import joblib
joblib.dump(final_pipe, 'final_model_pipeline.joblib')
joblib.dump(le, 'label_encoder.joblib')

print("Saved final pipeline and label encoder.")

# Discussion and Future Directions

The analysis in this notebook adopted a multi-faceted approach: after performing basic data cleaning and preprocessing, we conducted an exploratory analysis with several visualizations to assess the quality and distributions of the data. The RF classifier built to predict Disease Risk exhibited a certain level of accuracy, and further analysis such as the confusion matrix and ROC curve provide insights into the classifier's performance.

The merits of this approach include the comprehensive use of visualization techniques and a straightforward predictive model that is easy to interpret. In future analyses, one might consider:

- Using cross-validation and hyperparameter tuning to improve the predictive performance.
- Investigating additional feature engineering techniques, such as extracting k-mer patterns more robustly from the Sequence column.
- Analyzing the interplay between the sequence-specific features and other categorical attributes like Class_Label.
- Deploying more advanced models or ensemble methods to further refine predictions.

If you found these insights useful, please consider upvoting the notebook.